sentence_transformers1.py is a file equal to sentence_transformers.py in the official SBERT documentation. I just changed it to implement a callback that allows me to trigger EarlyStop on training, in case the loss is not decreasing for a user specified number of epochs.

In [ ]:
import torch
from sentence_transformers import InputExample, losses, util
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader
from torch import optim


import sys
sys.path.insert(0, '/home/SBERT_aux_files')
import evaluating
import sentence_transformers1

Load Data Folder - Where Main Data Files are

In [ ]:
data_folder = '/home/datasets/'
fine_tune_data = pd.read_csv(data_folder+ "fine_tune_Data.csv", encoding='utf-8')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Load model

In [ ]:
model = sentence_transformers1.SentenceTransformer('distiluse-base-multilingual-cased-v2', device=device)

<h3>Finetune the bi-encoder</h3>
Then save the model on folder, for reloading next time.

In [ ]:
train_examples = []

fine_tune_data = fine_tune_data.sample(frac=1, random_state=2319).reset_index(drop=True)

#Define your train examples.
for i in range(len(fine_tune_data)-1000):
    train_examples.append(InputExample(texts=[fine_tune_data.iloc[i,0], fine_tune_data.iloc[i,1]], label= int(fine_tune_data.iloc[i,2])))


evaluation_data = fine_tune_data.loc[len(fine_tune_data)-1000:len(fine_tune_data), ['Sa', 'Sb']]
labels_evaluation = fine_tune_data.iloc[len(fine_tune_data)-1000:len(fine_tune_data),2].tolist()

evaluator = evaluating.EmbeddingSimilarityEvaluator(
                sentences1 = evaluation_data.iloc[:,0].tolist(),
                sentences2 = evaluation_data.iloc[:,1].tolist(),
                scores = labels_evaluation,
                name = 'output',
                batch_size = 32,
                main_similarity = evaluating.SimilarityFunction.COSINE,
                show_progress_bar = True,
                write_csv = True
                )


train_dataloader = DataLoader(train_examples, shuffle=True, batch_size = 16) 
train_loss = losses.ContrastiveLoss(model = model, margin=0.5)

output_path = '/home/bi_encoder tuned'

#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], 
            evaluator=evaluator, 
            epochs=20, 
            steps_per_epoch=len(train_dataloader), # --> default <=> 1 Epoch = DataLoader size from train_objectives
            scheduler = 'WarmupLinear', # --> default <=> learning rate is increased from 0 up to the maximal learning rate.
            warmup_steps=100, # After these many training steps, the learning rate is decreased linearly back to zero.
            optimizer_class= optim.AdamW,
            optimizer_params={'lr': 1e-5},
            weight_decay= 0.01,
            output_path=output_path,
            save_best_model=True,
            show_progress_bar=True,
            callback = True,
            EarlyStop_Patience = 8,
            use_amp=True, # mixed precision for faster training
            )